In [1]:
import argparse
import logging
import subprocess

from agent.prompt import PROMPT
from agent.utils import process_repository

from dotenv import load_dotenv
import os

from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub
from langchain.tools import tool
from langchain.agents import initialize_agent
from langchain.tools import BaseTool

from typing import Union
from math import pi

In [4]:
class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius)*2.0*pi

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
    
class SimpleAgentV2:
    def __init__(self):
        self.api_key = self.get_api_key()
        self.llm = ChatOpenAI(openai_api_key=self.api_key, temperature=0, model="gpt-3.5-turbo")
        self.memory = ConversationBufferMemory(
                memory_key='chat_history',
                k=5,
                return_messages=True
        )
        
        self.tools = [CircumferenceTool()]

        self.agent = initialize_agent(
            agent='chat-conversational-react-description',
            tools=self.tools,
            llm=self.llm,
            verbose=True,
            max_iterations=3,
            early_stopping_method='generate',
            memory=self.memory
        )

    @staticmethod
    def get_api_key():
        load_dotenv()
        return os.getenv("OPENAI_API_KEY")
    
    def run_conversation(self, prompt):
        response = self.conv(prompt)
        return response
    
    def run(self, prompt):
        return self.agent(prompt)

# task_description = """
# I am running into an issue where the time my react page is 
# supposed to display is just 0 instead of the current timestamp. Fix this issue.
# """

# task_name = "fix_react_timestamp"
# use_mock = False

# repository_context = process_repository("/Users/inafi/Develop/Github/dave/apps/dave/example-task")

# agent_prompt = PROMPT.format(task_description=task_description, repository_context=repository_context)

agent = SimpleAgentV2()

agent.run("can you calculate the circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
```
Observation: 49.071677249072565
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}
```

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}